<a href="https://colab.research.google.com/github/Axieyan/TextReID_XY/blob/main/Text_Based_Person_Search_with_Limited_Data_Xie10_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 下载原代码


In [1]:
!git clone https://github.com/BrandonHanx/TextReID.git
%cd TextReID

Cloning into 'TextReID'...
remote: Enumerating objects: 368, done.
remote: Counting objects: 100% (141/141), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 368 (delta 115), reused 108 (delta 108), pack-reused 227
Receiving objects: 100% (368/368), 122.17 KiB | 10.18 MiB/s, done.
Resolving deltas: 100% (215/215), done.
/content/TextReID


# 配置环境


In [2]:
!pip install tensorflow tensorboardX

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 125 kB 28.1 MB/s 


In [3]:
!pip install -r requirements.txt
!pre-commit install

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 149 kB 27.4 MB/s 
     |████████████████████████████████| 496 kB 52.7 MB/s 
     |████████████████████████████████| 98 kB 6.2 MB/s 
     |████████████████████████████████| 15.7 MB 46.4 MB/s 
     |████████████████████████████████| 3.1 MB 57.1 MB/s 
     |████████████████████████████████| 191 kB 66.5 MB/s 
     |██████████████████████████████▎ | 834.1 MB 1.2 MB/s eta 0:00:41tcmalloc: large alloc 1147494400 bytes == 0x3906a000 @  0x7f77e4ed5615 0x58e046 0x4f2e5e 0x4d19df 0x51b31c 0x5b41c5 0x58f49e 0x51b221 0x5b41c5 0x58f49e 0x51837f 0x4cfabb 0x517aa0 0x4cfabb 0x517aa0 0x4cfabb 0x517aa0 0x4ba70a 0x538136 0x590055 0x51b180 0x5b41c5 0x58f49e 0x51837f 0x5b41c5 0x58f49e 0x51740e 0x58f2a7 0x517947 0x5b41c5 0x58f49e
     |████████████████████████████████| 881.9 MB 19 kB/s 
     |████████████████████████████████| 23.3 MB 1.3 MB/s 
     |██████████████████████

pre-commit installed at .git/hooks/pre-commit


In [4]:
!unzip '/content/drive/MyDrive/Subject1/datasets.zip' -d '/content/TextReID'

流式输出内容被截断，只能显示最后 5000 行内容。
  inflating: /content/TextReID/datasets/cuhkpedes/imgs/train_query/p6287_s8235.jpg  
  inflating: /content/TextReID/datasets/cuhkpedes/imgs/train_query/p6289_s8238.jpg  
  inflating: /content/TextReID/datasets/cuhkpedes/imgs/train_query/p6289_s8239.jpg  
  inflating: /content/TextReID/datasets/cuhkpedes/imgs/train_query/p6290_s8240.jpg  
  inflating: /content/TextReID/datasets/cuhkpedes/imgs/train_query/p6290_s8241.jpg  
  inflating: /content/TextReID/datasets/cuhkpedes/imgs/train_query/p6290_s8242.jpg  
  inflating: /content/TextReID/datasets/cuhkpedes/imgs/train_query/p6291_s8240.jpg  
  inflating: /content/TextReID/datasets/cuhkpedes/imgs/train_query/p6291_s8241.jpg  
  inflating: /content/TextReID/datasets/cuhkpedes/imgs/train_query/p6291_s8242.jpg  
  inflating: /content/TextReID/datasets/cuhkpedes/imgs/train_query/p6294_s8246.jpg  
  inflating: /content/TextReID/datasets/cuhkpedes/imgs/train_query/p6294_s8247.jpg  
  inflating: /content/TextReID/dataset

# 加载权重


In [5]:
!mkdir pretrained/clip/
!cd pretrained/clip
!wget https://openaipublic.azureedge.net/clip/models/afeb0e10f9e5a86da6080e35cf09123aca3b358a0c3e3b6c78a7b63bc04b6762/RN50.pt
!wget https://openaipublic.azureedge.net/clip/models/8fa8567bab74a42d41c5915025a8e4538c3bdbe8804a470a72f30b0d94fab599/RN101.pt
!cd -

mkdir: cannot create directory ‘pretrained/clip/’: No such file or directory
/bin/bash: line 0: cd: pretrained/clip: No such file or directory
--2022-10-08 14:41:47--  https://openaipublic.azureedge.net/clip/models/afeb0e10f9e5a86da6080e35cf09123aca3b358a0c3e3b6c78a7b63bc04b6762/RN50.pt
Resolving openaipublic.azureedge.net (openaipublic.azureedge.net)... 13.107.246.59, 13.107.213.59, 2620:1ec:bdf::59, ...
Connecting to openaipublic.azureedge.net (openaipublic.azureedge.net)|13.107.246.59|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 255827503 (244M) [application/octet-stream]
Saving to: ‘RN50.pt’

RN50.pt             100%[===================>] 243.98M   266MB/s    in 0.9s    

2022-10-08 14:41:48 (266 MB/s) - ‘RN50.pt’ saved [255827503/255827503]

--2022-10-08 14:41:48--  https://openaipublic.azureedge.net/clip/models/8fa8567bab74a42d41c5915025a8e4538c3bdbe8804a470a72f30b0d94fab599/RN101.pt
Resolving openaipublic.azureedge.net (openaipublic.azureedge.net)...

In [7]:
!unzip '/content/drive/MyDrive/Subject1/pretrained.zip' -d '/content/TextReID'

Archive:  /content/drive/MyDrive/Subject1/pretrained.zip
   creating: /content/TextReID/pretrained/
   creating: /content/TextReID/pretrained/clip/
  inflating: /content/TextReID/pretrained/clip/RN101.pt  
  inflating: /content/TextReID/pretrained/clip/RN50.pt  


In [8]:
%cd /content/TextReID
!python train_net.py \
--config-file configs/cuhkpedes/moco_gru_cliprn50_ls_bs128_2048.yaml \
--use-tensorboard

/content/TextReID
2022-10-08 14:47:24,090 PersonSearch INFO: Using 1 GPUs
2022-10-08 14:47:24,090 PersonSearch INFO: Namespace(config_file='configs/cuhkpedes/moco_gru_cliprn50_ls_bs128_2048.yaml', distributed=False, local_rank=0, opts=[], resume_from=None, root='./', use_tensorboard=True)
2022-10-08 14:47:24,090 PersonSearch INFO: Loaded configuration file configs/cuhkpedes/moco_gru_cliprn50_ls_bs128_2048.yaml
2022-10-08 14:47:24,090 PersonSearch INFO: 
MODEL:
  WEIGHT: "imagenet"
  FREEZE: False
  VISUAL_MODEL: "m_resnet50"
  TEXTUAL_MODEL: "bigru"
  NUM_CLASSES: 11003
  GRU:
    ONEHOT: "clip_vit"
    EMBEDDING_SIZE: 512
    NUM_UNITS: 512
    VOCABULARY_SIZE: 512
    DROPOUT_KEEP_PROB: 1.0
    MAX_LENGTH: 100
  RESNET:
    RES5_STRIDE: 1
  EMBEDDING:
    EMBED_HEAD: 'moco'
    FEATURE_SIZE: 256
    DROPOUT_PROB: 0.0
    EPSILON: 0.1
  MOCO:
    FC: False
    K: 2048
INPUT:
  HEIGHT: 384
  WIDTH: 128
  USE_AUG: True
  PIXEL_MEAN: [0.48145466, 0.4578275, 0.40821073]
  PIXEL_STD: [0.26

In [11]:
!cp /content/TextReID/output/cuhkpedes/moco_gru_cliprn50_ls_bs128_2048/best.pth /content/drive/MyDrive/Data

# 查看模型数据


In [10]:
!python test_net.py \
--config-file configs/cuhkpedes/moco_gru_cliprn50_ls_bs128_2048.yaml \
--checkpoint-file output/cuhkpedes/moco_gru_cliprn50_ls_bs128_2048/best.pth

loading annotations into memory...
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
2022-10-08 15:18:27,360 PersonSearch INFO: Using 1 GPUs
2022-10-08 15:18:27,360 PersonSearch INFO: AMP_VERBOSE: False
DATALOADER:
  EN_SAMPLER: True
  IMS_PER_ID: 4
  NUM_WORKERS: 4
DATASETS:
  TEST: ('cuhkpedes_test',)
  TRAIN: ('cuhkpedes_train',)
  USE_ONEHOT: True
DTYPE: float32
INPUT:
  HEIGHT: 384
  PADDING: 10
  PIXEL_MEAN: [0.48145466, 0.4578275, 0.40821073]
  PIXEL_STD: [0.26862954, 0.26130258, 0.27577711]
  USE_AUG: True
  WIDTH: 128
MODEL:
  DEVICE: cuda
  EMBEDDING:
    DROPOUT_PROB: 0.0
    